# Let's start the process by importing necessary modules

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LinearRegression

# Define the RMSE computation function

In [2]:
def compute_rmsle(y_test: np.ndarray, y_pred: np.ndarray, precision: int = 2) -> float:
    rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
    return round(rmsle, precision)

#Assigning data path for DATASETS
Data_path = '/home/kutty/Desktop/github project/dsp-kishorekumar-mourougane/data/'
Dataset_path = Data_path + 'house_train.csv'
# load the datasets
df_original = pd.read_csv(Dataset_path)
df = df_original.copy()
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


We need to select two continuous and two categorical features for our model. Let's take a closer look at the contents of the data, including details about the features and their types. We can get a better understanding of the data through summary statistics. Let's display this

In [3]:
# Display general information about the dataframe
df.info()

# Display summary statistics
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,1460.000000,1460.000000,1460,1201.000000,1460.000000,1460,91,1460,1460,1460,...,1460.000000,7,281,54,1460.000000,1460.000000,1460.000000,1460,1460,1460.000000
unique,NaN,NaN,5,NaN,NaN,2,2,4,4,2,...,NaN,3,4,4,NaN,NaN,NaN,9,6,NaN
top,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,...,NaN,Gd,MnPrv,Shed,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,NaN,1151,NaN,NaN,1454,50,925,1311,1459,...,NaN,3,157,49,NaN,NaN,NaN,1267,1198,NaN
mean,730.500000,56.897260,NaN,70.049958,10516.828082,NaN,NaN,NaN,NaN,NaN,...,2.758904,NaN,NaN,NaN,43.489041,6.321918,2007.815753,NaN,NaN,180921.195890
std,421.610009,42.300571,NaN,24.284752,9981.264932,NaN,NaN,NaN,NaN,NaN,...,40.177307,NaN,NaN,NaN,496.123024,2.703626,1.328095,NaN,NaN,79442.502883
min,1.000000,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,1.000000,2006.000000,NaN,NaN,34900.000000
25%,365.750000,20.000000,NaN,59.000000,7553.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,5.000000,2007.000000,NaN,NaN,129975.000000
50%,730.500000,50.000000,NaN,69.000000,9478.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,6.000000,2008.000000,NaN,NaN,163000.000000
75%,1095.250000,70.000000,NaN,80.000000,11601.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,8.000000,2009.000000,NaN,NaN,214000.000000


let's use 'GrLivArea' and 'LotArea' as continuous features, and 'Neighborhood' and 'MSZoning' as categorical features. 'GrLivArea' and 'LotArea' are numerical and represent areas of the house which can be important for predicting the house price, whereas 'Neighborhood' and 'MSZoning' are nominal and represent geographical and zoning classifications, respectively. Let's now create a new DataFrame with only these selected features along with our target feature, 'SalePrice'. 

In [4]:
# Selecting The Features
features = ['GrLivArea', 'LotArea', 'Neighborhood', 'MSZoning']
target = ['SalePrice']

# Creating A New DataFrame With Selected Features
df_selected = df[features + target]

# Removing Missing Values
df_selected = df_selected.dropna()

# Display the first few rows of the new dataframe
df_selected.head()

,GrLivArea,LotArea,Neighborhood,MSZoning,SalePrice
0,1710,8450,CollgCr,RL,208500
1,1262,9600,Veenker,RL,181500
2,1786,11250,CollgCr,RL,223500
3,1717,9550,Crawfor,RL,140000
4,2198,14260,NoRidge,RL,250000


# Let's start by splitting the dataset into a train and test set. 
We'll use a standard split of 70% for training and 30% for testing. After that, we'll perform the scaling

In [5]:
# Splitting the dataset into train and test sets
train_df, test_df = train_test_split(df_selected, test_size=0.3, random_state=42)

# Separating Features and the Target Variable For Train and Test sets
X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]


In [6]:
# Scaling Continuous Variables
scaler = StandardScaler()

# Fit on training set
scaler.fit(X_train[['GrLivArea', 'LotArea']])

# Transform both training and test sets
X_train.loc[:, ['GrLivArea', 'LotArea']] = scaler.transform(X_train[['GrLivArea', 'LotArea']])
X_test.loc[:, ['GrLivArea', 'LotArea']] = scaler.transform(X_test[['GrLivArea', 'LotArea']])

# One-hot Encoding for Categorical Variables
encoder = OneHotEncoder(drop='first', sparse_output=False)

For numerical features, we'll use the StandardScaler which standardizes the features by removing the mean and scaling to unit variance. 

For categorical features, let's use OneHotEncoder which creates binary features for each category/label present in the column.

In [7]:
# Fit on training set
encoder.fit(X_train[['Neighborhood', 'MSZoning']])

# Transform both training and test sets
encoded_features_train = encoder.transform(X_train[['Neighborhood', 'MSZoning']])
encoded_features_train = pd.DataFrame(encoded_features_train, columns=encoder.get_feature_names_out(['Neighborhood', 'MSZoning']), index=X_train.index)
X_train = pd.concat([X_train, encoded_features_train], axis=1).drop(['Neighborhood', 'MSZoning'], axis=1)

encoded_features_test = encoder.transform(X_test[['Neighborhood', 'MSZoning']])
encoded_features_test = pd.DataFrame(encoded_features_test, columns=encoder.get_feature_names_out(['Neighborhood', 'MSZoning']), index=X_test.index)
X_test = pd.concat([X_test, encoded_features_test], axis=1).drop(['Neighborhood', 'MSZoning'], axis=1)

X_train.head()

,GrLivArea,LotArea,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,...,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM
135,0.287835,-0.030504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1452,-0.861565,-0.624365,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
762,0.033459,-0.185924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
932,0.708025,0.081645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
435,0.248265,-0.006927,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


The numerical features are scaled, and the categorical features have been transformed using one-hot encoding.
# Train model using Linear Regression

In [8]:
# Initialize the Linear Regression Model
model = LinearRegression()

# Fit the Model on the Training Data
model.fit(X_train, y_train)

# Predict on the Test Set
y_pred = model.predict(X_test)

# Show the first few predictions
y_pred[:5]

array([[125676.07828921],
       [336436.75612153],
       [ 93902.20426864],
       [157887.19010127],
       [221516.06689756]])

The results are the predicted 'SalePrice' for the given house features in the test data.
# The Root-Mean-Squared-Error (RMSE)
between the logarithm of the predicted value and the logarithm of the observed sales price.

In [9]:
# Evaluate the Model
rmsle = compute_rmsle(y_test, y_pred)
rmsle

0.19

observed sales prices in both the training and validation sets is approximately 0.19.

# Testing dataframe equality

In [10]:
# Save the dataframes as .parquet
train_df.to_parquet(Data_path + 'train_df.parquet', index=False)
test_df.to_parquet(Data_path + 'test_df.parquet', index=False)

# Load the saved dataframes
train_df_loaded = pd.read_parquet(Data_path + 'train_df.parquet')
test_df_loaded = pd.read_parquet(Data_path + 'test_df.parquet')

# Check that the loaded dataframes are equal to the original
pd.testing.assert_frame_equal(train_df, train_df_loaded)
pd.testing.assert_frame_equal(test_df, test_df_loaded)

AssertionError: DataFrame.index are different

DataFrame.index values are different (99.90215 %)
[left]:  Index([ 135, 1452,  762,  932,  435,  629, 1210, 1118, 1084,  158,
       ...
        330, 1238,  466,  121, 1044, 1095, 1130, 1294,  860, 1126],
      dtype='int64', length=1022)
[right]: RangeIndex(start=0, stop=1022, step=1)
At positional index 0, first diff: 135 != 0

It seems that there was a problem with comparing our saved and loaded data. The issue is the inconsistency in index between the saved and loaded dataframes. This happened because the index was reset during the train-test split and was not saved when writing to .parquet file.

To solve this, I will reset the index before saving the dataframe. Let's try to save, load, and compare the dataframes again after resetting the index.

In [11]:
# Reset index of the dataframes before saving
train_df_reset = train_df.reset_index(drop=True)
test_df_reset = test_df.reset_index(drop=True)

# Save the dataframes as .parquet again
train_df_reset.to_parquet(Data_path + 'train_df.parquet', index=False)
test_df_reset.to_parquet(Data_path + 'test_df.parquet', index=False)

# Load the saved dataframes
train_df_loaded = pd.read_parquet(Data_path + 'train_df.parquet')
test_df_loaded = pd.read_parquet(Data_path + 'test_df.parquet')

# Check that the loaded dataframes are equal to the original
pd.testing.assert_frame_equal(train_df_reset, train_df_loaded)
pd.testing.assert_frame_equal(test_df_reset, test_df_loaded)

Finally, we confirmed the reproducibility of our data processing by saving and reloading the processed data.